# Anzeige der Temperaturdaten aus der Heizung aus Temperaturen.db

In [31]:
#!scp 192.168.202.41:/u1/HeizungTemperaturen/Temperaturen.db .
import paramiko
import ipywidgets as widgets
import datetime
import sqlite3
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.options.plotting.backend = "plotly"
pd.options.mode.chained_assignment = None 
#import plotly.express as px*
import os, sys
import ipywidgets as widgets
import matplotlib.pyplot as pl

from scp import SCPClient
DB_FILENAME = 'Temperaturen.db'
def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client
def get_db_file():
    ssh = createSSHClient("192.168.202.41", 22, "root", "sni")
    scp = SCPClient(ssh.get_transport())
    scp.get('/var/lib/grafana/' + DB_FILENAME)


layout_btn = widgets.Layout(width='200px')
button = widgets.Button(description="Temperaturdaten holen", layout=layout_btn)
button.style.button_color = '#90ee90'
output = widgets.Output()
display(button, output)

def on_button_clicked():
    get_db_file()

Button(description='Temperaturdaten holen', layout=Layout(width='200px'), style=ButtonStyle(button_color='#90e…

Output()

In [28]:

def get_df(db_filename):
    db_file = db_filename
    
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = os.path.join('/var/lib/grafana', db_filename)
    elif os.path.exists(os.path.join('.', db_filename)):
        db_file = os.path.join('.', db_filename)    
    else:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
    # Blanks mit np.nan ersetzen
    return df.replace(r'^\s*$', np.nan, regex=True)
    
def plot_dfplot(start, dauer, einheit):
    # Mit Dropdown: datetime.datetime
    # Mit DatePickier: datetime.date

    #start_object = datetime.datetime.strptime(start, '%Y-%m-%d')
    start_object = datetime.datetime(start.year, start.month, start.day)
    
    if einheit == 'Tage':
        end_date = start_object + datetime.timedelta(days=dauer)
    elif einheit == 'Stunden':
        end_date = start_object + datetime.timedelta(hours=dauer)
    else:
        end_date = start_object + datetime.timedelta(days=dauer)
    #print('Start: ' + str(start) + ' End: ' + str(end_date))
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_date )]
    #df = df['date'] + pd.DateOffset(months=plus_month_period)
    #data.info()
    # UnixTime 	AussenTemp 	VorlaufTemp 	RuecklaufTemp 	WohnzimmerTemp
    subset.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"], grid=True)

    


In [16]:
def plot_plotly(start, dauer, einheit):
    # Mit Dropdown: datetime.datetime
    # Mit DatePickier: datetime.date

    #start_object = datetime.datetime.strptime(start, '%Y-%m-%d')
    start_object = datetime.datetime(start.year, start.month, start.day)
    
    if einheit == 'Tage':
        end_date = start_object + datetime.timedelta(days=dauer)
    elif einheit == 'Stunden':
        end_date = start_object + datetime.timedelta(hours=dauer)
    else:
        end_date = start_object + datetime.timedelta(days=dauer)
    #print('Start: ' + str(start) + ' End: ' + str(end_date))
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_date )]
    
    subset.set_index(['UnixTime'])
    #df = df['date'] + pd.DateOffset(months=plus_month_period)
    #data.info()
    # UnixTime 	AussenTemp 	VorlaufTemp 	RuecklaufTemp 	WohnzimmerTemp
    #subset.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"])
    #fig = subset.plot(x='UnixTime',y='VorlaufTemp')
    subset = subset.astype({'WohnzimmerTemp':'float64'})
    subset = subset.astype({'AussenTemp':'float64'})    
    fig = subset.plot(x='UnixTime',y=['VorlaufTemp', 'RuecklaufTemp', 'WohnzimmerTemp', 'AussenTemp'], title="Temperaturverlauf Heizung",
                     labels={
                     "UnixTime": "Zeit",
                     "VorlaufTemp": "VorlaufTemperatur",
                     "species": "Species of Iris"
                 },)
    fig.update_layout(
    xaxis_title="Zeit", yaxis_title="Temperaturen in °C"
)
    fig.show()
 
# main
try:    
    df= get_df(DB_FILENAME)
except SystemExit as _e:
    print(_e)
    sys.exit(-1)
    
df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s')
df.set_index(['UnixTime'])


dates_us = df.UnixTime.dt.strftime('%Y-%m-%d').unique()
#print(dates_us[0])
#df['Aussen'] = str(df['AussenTemp'])
#df['Wohnzimmer'] = str(df['WohnzimmerTemp'])

widget_dauer = widgets.IntSlider(min=1,max=10,step=1, description="Dauer", value=3)
widget_start = widgets.Select(options=np.sort(dates_us)[::-1], rows=1,description='Startdatum:')
widget_start_picker = widgets.DatePicker(description='Beginn Zeitraum', value = datetime.datetime.now().date() - datetime.timedelta(days=4))
widget_einheit = widgets.Select(options=['Stunden','Tage'], rows=1, value='Tage', continous_update=True)
widgets.interact(plot_plotly, start= widget_start_picker, dauer = widget_dauer, einheit=widget_einheit);

interactive(children=(DatePicker(value=datetime.date(2023, 10, 16), description='Beginn Zeitraum', step=1), In…